# SI 630: Homework 1 - Classification  (Sangho Eum)

## TASK 2: Logistic Regression

## Conclusion

- Given some choice of β to make predictions, we want to use the difference in our prediction Yˆ from the ground truth Y to update β. The gradient of the log likelihood tells us which direction (positive or negative) to make the update and how large the update should be. Write a function compute gradient to compute the gradient. Note that we can compute the whole gradient using 
∇ l l = X T ( Y − Yˆ )
Note that Y is a binary vector with our ground truth (i.e., the training data labels) and Yˆ is
the binary vector with the predictions. To get a sense of why this works, think about what
gradient will equal if our prediction for item i, Yˆ is the same as the ground truth Y ; if we ii
use this gradient to update our weight for βi, what effect will it have?

-> Gradient is zero, so it does not make an impact. 

- Train your model on the training data learning rate=5e-5 (i.e., a very small number) and num steps = 300000. Make a plot of the log-likelihood every 10,000 steps. Did the model converge at some point (i.e., does the log likelihood remain stable)?

-> Yes, it converges to some point. It will be getting more stable.

- Change the learning rate to a much larger and much smaller value and repeat the train- ing procedure for each. Plot all three curves together and describe what you observe. You’re welcome (encouraged, even!) to try additional learning rates. If your model is very slow or if it converges quickly, you can also reduce the number of steps for this question.

-> See my graph below. I created curves with learning rate 5e-3, 5e-5, 5e-7 respectively.

- After training on the training data, use your logistic regression classifier to make predictions on the validation dataset and report your performance using the F1.

-> 0.9600 with 5e-5 learning rate

- Submit your best model’s predictions on the test data to Kaggle.

-> I have submitted mine and score is 0.95979


In [2]:
# import library
import csv
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy.sparse import csr_matrix, hstack
import logging, sys
import math
from sklearn.naive_bayes import GaussianNB

In [3]:
# load data
dat = []
dev = []
stop_words = []
with open('train.tsv') as tsvfile:
  reader = csv.DictReader(tsvfile, dialect='excel-tab', quoting=csv.QUOTE_NONE)
  for i in reader:
      dat.append(i)

with open('dev.tsv') as tsvfile:
  reader2 = csv.DictReader(tsvfile, dialect='excel-tab', quoting=csv.QUOTE_NONE)
  for i in reader2:
      dev.append(i)

with open('stopwords.txt', 'r') as file:
    for line in file.readlines():
        stop_words.append(line.strip())


In [4]:
# tokenize % better tokenize functions
def tokenize(text):
    return text.split()

def better_tokenize(text):
    text = text.lower()
    text = re.sub(r"(@[a-zA-Z0-9_]+)|(\w+:\/\/\S+)", " ", text)
    text = re.sub('[^A-Za-z0-9 ]+', '', text)
    features = text.split()
    filtered_features = []
    for feature in features:
        if feature not in stop_words:
            filtered_features.append(feature)
    return filtered_features

In [5]:
# Create matrix function
def create_matrix(lst):
    indptr = [0]
    indices = []
    data = []
    vocabulary = {}
    y_true = []

    for i in lst:
        y_true.append(int(i['class']))

        text = better_tokenize(i['text'])
        for k in text:
            index = vocabulary.setdefault(k, len(vocabulary))
            indices.append(index)
            data.append(1)
        indptr.append(len(indices))
    a = csr_matrix((data, indices, indptr)).toarray()
    return [a, np.array([y_true]).T, vocabulary]

In [184]:
# Create matrix with train.tsv data set 
train_set = create_matrix(dat)
train_x = train_set[0]
train_y = train_set[1]
train_v = train_set[2]


sample_x=[]
sample_y = []

for i in range(train_x.shape[0]):
    sample_x.append(train_x[i])
    
sample_x=np.array(sample_x)

for i in train_y:
    sample_y.append(i)    

sample_y = np.array(sample_y).ravel()

In [185]:
clf = GaussianNB()
clf.fit(sample_x, sample_y)

GaussianNB(priors=None)

In [186]:
from sklearn.metrics import accuracy_score

In [187]:
def predict(d, vocabulary, label = True, add_intercept = False):
    yp = []
    yt = []
    inst =[]
    xt= []

    for i in d:
        inst.append(i['instance_id'])
        if label:
            yt.append(int(i['class']))


        if add_intercept:
            x = np.zeros((1, len(vocabulary)+1))
            x[0, 0] = 1
            text = better_tokenize(i['text'])
            for k in text:
                if k in vocabulary:
                    index = vocabulary[k]
                    x[0, index+1] += 1
            xt.append(x)
        else:
            x = np.zeros((1, len(vocabulary)))
            text = better_tokenize(i['text'])
            for k in text:
                if k in vocabulary:
                    index = vocabulary[k]
                    x[0, index] += 1
            xt.append(x)

    return {'x': xt, 'yt': yt, 'inst_id': inst}

In [188]:
sample_set = predict(dev, train_v, True, False)

In [189]:
test_x = sample_set['x']
test_yt = sample_set['yt']


print(test_x[0].shape)
sample_x1 =[]
sample_yt1 = []

for i in range(len(test_x)):     
    sample_x1.append(test_x[i][0])
sample_x1= np.array(sample_x1)

for i in test_yt:
    sample_yt1.append(i)

sample_yt1 = np.array(sample_yt1).ravel()

(1, 38010)


In [190]:
sample_yp1 = clf.predict(sample_x1)
print(sample_yp1)

[1 1 0 ..., 1 1 1]


In [191]:
accuracy_score(sample_yp1, sample_yt1)

0.66324251898168829